## Transform Orders Data - String to JSON
1. Pre-process the JSON String to fix the Data Quality Issues
1. Transform JSON String to JSON Object
1. Write transformed data to the silver schema

In [0]:
-- 1. 
select *
from gizmobox.bronze.v_orders

In [0]:
create or replace temp view tv_orders_fixed
as 
select
  value,
  regexp_replace(value, '"order_date": (\\d{4}-\\d{2}-\\d{2})', '"order_date": "\$1"')as fixed_value
  from gizmobox.bronze.v_orders

In [0]:
select * FROM tv_orders_fixed

In [0]:
-- 2.

select
  schema_of_json(fixed_value) as schema,
  fixed_value
from tv_orders_fixed
limit 1

In [0]:
select
  from_json(fixed_value,
                  'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>' ) as json_value,
  fixed_value
from tv_orders_fixed
limit 1



In [0]:
-- 3. 

CREATE TABLE IF NOT EXISTS gizmobox.silver.orders_json
AS
select
  from_json(fixed_value,
                  'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>' ) as json_value
from tv_orders_fixed

In [0]:
select * from gizmobox.silver.orders_json